# **Comparing the low-level API function *loadproblem()* with high-level API functions in model building time performance**

***load_problem.ipynb***

This example uses the *time* package to measure the differences in model building performance between the low-level API [p.loadproblem()](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/python/HTML/problem.loadproblem.html) and the higher level API functions [p.addConstraint()](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/python/HTML/problem.addConstraint.html) and [p.setObjective()](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/python/HTML/problem.setObjective.html) for a randomly created problem.

&copy; Copyright 2025 Fair Isaac Corporation

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0.
 
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

This example uses FICO&reg; Xpress software. By running it, you agree to the Community License terms of the [Xpress Shrinkwrap License Agreement](https://community.fico.com/s/contentdocument/06980000002h0i5AAA) with respect to the FICO&reg; Xpress software. See the [licensing options](https://www.fico.com/en/fico-xpress-trial-and-licensing-options) overview for additional details and information about obtaining a paid license.

In [ ]:
# Install the xpress package
%pip install -q xpress

The [p.loadproblem()](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/python/HTML/problem.loadproblem.html) function provides a low-level interface to the FICO&reg; Xpress Optimizer libraries. It is the preferable option for very large problems and when efficiency in model creation is crucial, although it **may require extra modeling effort, and it is typically harder to maintain**. The low-level interface can be used to create problems with linear/quadratic constraints, a linear/quadratic objective function, and with continuous/discrete variables.

Other API functions such as [p.addrows()](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/python/HTML/problem.addrows.html), [p.addcols()](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/python/HTML/problem.addcols.html) or [p.addpwlcons()](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/python/HTML/problem.addpwlcons.html) can also be used at the low level to enhance efficiency in model building performance.

## Data generation

Start by importing the necessary packages and create randomly sampled data for the problem with **1 million variables and constraints**. 

Read the comments after each data object creation for a description of the arguments needed for the low-level API function.

In [1]:
import xpress as xp
import numpy as np
import time
import random

rndseed = 10
random.seed(rndseed)

# Variables and Constraints
V=1000000
C=1000000

# Generate random data
rowtype = ['G' for i in range(C)]                           # Character array containing row types
rhs = [random.randint(0, 100) for _ in range(C)]            # Right hand side coefficients of the rows
obj = np.array([random.randint(0, V) for i in range(V)])    # Objective function coefficients
start = [s*2 for s in range(V+1)]                           # Offsets in the rowind and rowcoef arrays of the start of the elements for each column
rowind = [j for _ in range(2*(int(V/C))) for j in range(C)] # Row indices for the nonzero elements
rowcoef = [random.randint(1,5) for _ in range(start[-1])]   # Nonzero coefficients; length as for rowind
lb = [0 for _ in range(V)]                                  # Lower bound
ub = [random.randint(50, 100) for _ in range(V)]            # Upper bound

## Low-level API with [p.loadproblem()](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/python/HTML/problem.loadproblem.html)

Now use the low-level API [p.loadproblem()](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/python/HTML/problem.loadproblem.html) to build the model, measuring the total time taken by recording a time stamp right before and after the call to the function.

In [ ]:
# LOW LEVEL API
p = xp.problem("low-level")

start_t = time.time() # records start time
p.loadproblem("lowlevel",
              rowtype,
              rhs,
              None,
              obj,
              start,
              None,
              rowind,
              rowcoef,
              lb,
              ub)
end_t = time.time() # record end time
print(f"\nModel building time with low-level api: {(end_t-start_t)} secs.")

# p.write("lowlevel", "lp")
# p.optimize()
# print("Objective value w/ low-level api:", p.attributes.objval)

## High-level API functions

The cell below builds exactly the same model, but using the higher level API functions [p.addConstraint()](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/python/HTML/problem.addConstraint.html) and [p.setObjective()](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/python/HTML/problem.setObjective.html), after getting the proper expressions from the same data objects.

The time needed to extract the data from the previous data objects (and are not part of the model building process) is excluded from the time recording, which only includes creation of variables, constraints and objective.

Run the code cell below and compare the time taken to build the model with the time previously recorded for the low-level API function.

In [ ]:
# HIGH LEVEL API
p = xp.problem("high-level") # re-set problem

start_t = time.time() # records start time
x = np.array([p.addVariable(lb=0, ub=ub[i]) for i in range(V)])
end_t = time.time() # record end time
time1 = end_t-start_t

# Get constraint expressions from above data
expr = [0 for _ in range(C)]
cnt = 0
for j in start[:len(start)-1]:
    for i in range(start[cnt],start[cnt+1]):
        expr[rowind[i]] += rowcoef[i] * x[cnt]
    cnt += 1

start_t = time.time() # records start time
p.addConstraint(expr[i] >= rhs[i] for i in range(C))
p.setObjective(xp.Dot(obj,x))
end_t = time.time() # record end time
time2 = end_t-start_t
print(f"Model building time with high-level api: {(time1+time2)} secs.")

# p.write("highlevel", "lp")
# p.optimize()
# print("Objective value w/ high-level api:", p.attributes.objval)